In [ ]:
# Imports
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import json
from collections import Counter
import warnings
warnings.filterwarnings('ignore')

# Configuration graphiques
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette("husl")
%matplotlib inline

print("✓ Imports réussis !")

## 1️⃣ Chargement des Données

In [ ]:
# Charger les datasets
train_df = pd.read_csv('../data/processed/train.csv')
val_df = pd.read_csv('../data/processed/val.csv')
test_df = pd.read_csv('../data/processed/test.csv')

print(f"📁 Données chargées :")
print(f"   Train : {len(train_df)} exemples")
print(f"   Val   : {len(val_df)} exemples")
print(f"   Test  : {len(test_df)} exemples")
print(f"   Total : {len(train_df) + len(val_df) + len(test_df)} exemples")

In [ ]:
# Aperçu des données
print("\n📋 Aperçu des 5 premières lignes :")
train_df.head()

In [ ]:
# Informations sur les colonnes
print("\nℹ️ Informations sur le dataset :")
train_df.info()

## 2️⃣ Statistiques Descriptives

In [ ]:
# Longueur des textes
train_df['text_length'] = train_df['text'].str.len()
train_df['word_count'] = train_df['text'].str.split().str.len()

print("📏 Statistiques sur les textes :")
print(f"\nLongueur en caractères :")
print(train_df['text_length'].describe())
print(f"\nNombre de mots :")
print(train_df['word_count'].describe())

In [ ]:
# Distribution des labels
print("\n🏷️ Distribution des labels :\n")

print("Émotions :")
print(train_df['emotion'].value_counts())

print("\nSentiment :")
sentiment_map = {0: 'Négatif', 1: 'Neutre', 2: 'Positif'}
print(train_df['sentiment'].map(sentiment_map).value_counts())

print("\nIronie :")
irony_map = {0: 'Non-ironique', 1: 'Ironique'}
print(train_df['is_ironic'].map(irony_map).value_counts())

## 3️⃣ Visualisations

In [ ]:
# Distribution des longueurs de texte
fig, axes = plt.subplots(1, 2, figsize=(15, 5))

# Histogramme des longueurs
axes[0].hist(train_df['text_length'], bins=50, color='skyblue', edgecolor='black')
axes[0].set_title('Distribution des Longueurs de Texte (caractères)', fontsize=14)
axes[0].set_xlabel('Longueur (caractères)')
axes[0].set_ylabel('Fréquence')
axes[0].axvline(train_df['text_length'].mean(), color='red', linestyle='--', label=f'Moyenne: {train_df["text_length"].mean():.0f}')
axes[0].legend()

# Histogramme du nombre de mots
axes[1].hist(train_df['word_count'], bins=30, color='lightgreen', edgecolor='black')
axes[1].set_title('Distribution du Nombre de Mots', fontsize=14)
axes[1].set_xlabel('Nombre de mots')
axes[1].set_ylabel('Fréquence')
axes[1].axvline(train_df['word_count'].mean(), color='red', linestyle='--', label=f'Moyenne: {train_df["word_count"].mean():.0f}')
axes[1].legend()

plt.tight_layout()
plt.show()

In [ ]:
# Distribution des classes (3 tâches)
fig, axes = plt.subplots(1, 3, figsize=(18, 5))

# Émotions
emotion_counts = train_df['emotion'].value_counts()
axes[0].bar(emotion_counts.index, emotion_counts.values, color='coral')
axes[0].set_title('Distribution des Émotions', fontsize=14, fontweight='bold')
axes[0].set_xlabel('Émotion')
axes[0].set_ylabel('Nombre d\'exemples')
axes[0].tick_params(axis='x', rotation=45)

# Sentiment
sentiment_counts = train_df['sentiment'].map(sentiment_map).value_counts()
axes[1].bar(sentiment_counts.index, sentiment_counts.values, color='skyblue')
axes[1].set_title('Distribution des Sentiments', fontsize=14, fontweight='bold')
axes[1].set_xlabel('Sentiment')
axes[1].set_ylabel('Nombre d\'exemples')

# Ironie
irony_counts = train_df['is_ironic'].map(irony_map).value_counts()
axes[2].bar(irony_counts.index, irony_counts.values, color='lightgreen')
axes[2].set_title('Distribution de l\'Ironie', fontsize=14, fontweight='bold')
axes[2].set_xlabel('Type')
axes[2].set_ylabel('Nombre d\'exemples')

plt.tight_layout()
plt.show()

In [ ]:
# Camembert (pie chart) pour mieux voir les proportions
fig, axes = plt.subplots(1, 3, figsize=(18, 5))

# Émotions
axes[0].pie(emotion_counts.values, labels=emotion_counts.index, autopct='%1.1f%%', startangle=90)
axes[0].set_title('Proportion des Émotions', fontsize=14, fontweight='bold')

# Sentiment
axes[1].pie(sentiment_counts.values, labels=sentiment_counts.index, autopct='%1.1f%%', startangle=90, colors=['#ff9999','#66b3ff','#99ff99'])
axes[1].set_title('Proportion des Sentiments', fontsize=14, fontweight='bold')

# Ironie
axes[2].pie(irony_counts.values, labels=irony_counts.index, autopct='%1.1f%%', startangle=90, colors=['#ffcc99', '#ff9999'])
axes[2].set_title('Proportion de l\'Ironie', fontsize=14, fontweight='bold')

plt.tight_layout()
plt.show()

## 4️⃣ Analyse des Déséquilibres de Classes

**Important** : Les déséquilibres peuvent affecter les performances du modèle !

In [ ]:
# Calculer le déséquilibre
print("⚖️ Analyse des déséquilibres :\n")

# Émotions
emotion_balance = emotion_counts.max() / emotion_counts.min()
print(f"Émotions :")
print(f"  Ratio max/min : {emotion_balance:.2f}x")
if emotion_balance > 3:
    print(f"  ⚠️ Déséquilibre important ! Considérer class weighting ou oversampling")
else:
    print(f"  ✓ Déséquilibre acceptable")

# Sentiment
sentiment_balance = sentiment_counts.max() / sentiment_counts.min()
print(f"\nSentiment :")
print(f"  Ratio max/min : {sentiment_balance:.2f}x")
if sentiment_balance > 3:
    print(f"  ⚠️ Déséquilibre important !")
else:
    print(f"  ✓ Déséquilibre acceptable")

# Ironie
irony_balance = irony_counts.max() / irony_counts.min()
print(f"\nIronie :")
print(f"  Ratio max/min : {irony_balance:.2f}x")
if irony_balance > 2:
    print(f"  ⚠️ Déséquilibre important !")
else:
    print(f"  ✓ Déséquilibre acceptable")

## 5️⃣ Exemples de Textes par Catégorie

In [ ]:
# Afficher des exemples pour chaque émotion
print("📝 Exemples de textes par émotion :\n")

for emotion in train_df['emotion'].unique():
    print(f"\n{'='*60}")
    print(f"ÉMOTION : {emotion.upper()}")
    print(f"{'='*60}")
    
    examples = train_df[train_df['emotion'] == emotion].sample(n=min(3, len(train_df[train_df['emotion'] == emotion])))
    
    for idx, row in examples.iterrows():
        print(f"\n  • {row['text']}")
        print(f"    → Sentiment: {sentiment_map[row['sentiment']]}, Ironie: {irony_map[row['is_ironic']]}")

## 6️⃣ Corrélations entre les Tâches

In [ ]:
# Analyser la corrélation entre émotion et sentiment
print("🔗 Corrélations Émotion ↔ Sentiment :\n")

cross_tab = pd.crosstab(train_df['emotion'], train_df['sentiment'].map(sentiment_map))
print(cross_tab)

# Visualiser
plt.figure(figsize=(10, 6))
sns.heatmap(cross_tab, annot=True, fmt='d', cmap='YlOrRd')
plt.title('Heatmap : Émotion vs Sentiment', fontsize=14, fontweight='bold')
plt.xlabel('Sentiment')
plt.ylabel('Émotion')
plt.show()

In [ ]:
# Analyser la corrélation entre ironie et sentiment
print("\n🔗 Corrélations Ironie ↔ Sentiment :\n")

cross_tab2 = pd.crosstab(train_df['is_ironic'].map(irony_map), train_df['sentiment'].map(sentiment_map))
print(cross_tab2)

# Visualiser
plt.figure(figsize=(8, 5))
sns.heatmap(cross_tab2, annot=True, fmt='d', cmap='Blues')
plt.title('Heatmap : Ironie vs Sentiment', fontsize=14, fontweight='bold')
plt.xlabel('Sentiment')
plt.ylabel('Ironie')
plt.show()

## 7️⃣ Analyse de la Ponctuation et Emojis

**Important** : Les emojis et la ponctuation sont cruciaux pour l'analyse de sentiment !

In [ ]:
import re

# Compter les emojis
def count_emojis(text):
    emoji_pattern = re.compile("["
        u"\U0001F600-\U0001F64F"  # emoticons
        u"\U0001F300-\U0001F5FF"  # symbols & pictographs
        u"\U0001F680-\U0001F6FF"  # transport & map symbols
        u"\U0001F1E0-\U0001F1FF"  # flags
        u"\U00002702-\U000027B0"
        u"\U000024C2-\U0001F251"
        "]+", flags=re.UNICODE)
    return len(emoji_pattern.findall(text))

train_df['emoji_count'] = train_df['text'].apply(count_emojis)
train_df['exclamation_count'] = train_df['text'].str.count('!')
train_df['question_count'] = train_df['text'].str.count('\?')

print("😊 Statistiques sur les emojis et ponctuation :\n")
print(f"Textes avec emojis : {(train_df['emoji_count'] > 0).sum()} ({(train_df['emoji_count'] > 0).sum() / len(train_df) * 100:.1f}%)")
print(f"Textes avec '!' : {(train_df['exclamation_count'] > 0).sum()} ({(train_df['exclamation_count'] > 0).sum() / len(train_df) * 100:.1f}%)")
print(f"Textes avec '?' : {(train_df['question_count'] > 0).sum()} ({(train_df['question_count'] > 0).sum() / len(train_df) * 100:.1f}%)")

## 8️⃣ Sauvegarde du Rapport d'Exploration

In [ ]:
# Créer un rapport récapitulatif
report = {
    'dataset_size': {
        'train': len(train_df),
        'val': len(val_df),
        'test': len(test_df),
        'total': len(train_df) + len(val_df) + len(test_df)
    },
    'text_statistics': {
        'avg_length_chars': float(train_df['text_length'].mean()),
        'avg_words': float(train_df['word_count'].mean()),
        'max_length': int(train_df['text_length'].max()),
        'min_length': int(train_df['text_length'].min())
    },
    'class_distribution': {
        'emotions': emotion_counts.to_dict(),
        'sentiment': sentiment_counts.to_dict(),
        'irony': irony_counts.to_dict()
    },
    'class_balance': {
        'emotion_ratio': float(emotion_balance),
        'sentiment_ratio': float(sentiment_balance),
        'irony_ratio': float(irony_balance)
    },
    'special_features': {
        'texts_with_emojis': int((train_df['emoji_count'] > 0).sum()),
        'texts_with_exclamation': int((train_df['exclamation_count'] > 0).sum()),
        'texts_with_question': int((train_df['question_count'] > 0).sum())
    }
}

# Sauvegarder
with open('../data/exploration_report.json', 'w', encoding='utf-8') as f:
    json.dump(report, f, indent=2, ensure_ascii=False)

print("✅ Rapport d'exploration sauvegardé dans 'data/exploration_report.json'")

## ✅ Conclusions

**Points clés identifiés** :

1. **Taille du dataset** : Suffisant pour un entraînement initial (~1500 exemples)
2. **Déséquilibres** : À surveiller, peut nécessiter class weighting
3. **Longueur des textes** : Courts à moyens (typique des tweets/commentaires)
4. **Caractéristiques spéciales** : Présence d'emojis et ponctuation expressive

**Prochaines étapes** :
- ✅ Phase 1 : Exploration → **TERMINÉE**
- 🔜 Phase 2 : Preprocessing (nettoyage, tokenization)
- 🔜 Phase 3 : Modèle baseline
- 🔜 Phase 4 : CamemBERT multi-tâches